Timm is pytorch image models at : https://github.com/huggingface/pytorch-image-models

In [1]:
!pip install timm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 34.3 MB/s eta 0:00:00


In [7]:
url = "https://www.kaggle.com/datasets/likhon148/animal-data"

In [3]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d likhon148/animal-data

Dataset URL: https://www.kaggle.com/datasets/likhon148/animal-data
License(s): MIT
 91% 33.0M/36.3M [00:02<00:00, 27.8MB/s]
100% 36.3M/36.3M [00:02<00:00, 17.4MB/s]


In [8]:
# Import files
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms, models
from PIL import Image
from tqdm.auto import tqdm
import time
import timm

In [9]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cpu'

In [10]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [11]:
!unzip -q animal-data.zip

In [12]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Gather file paths of all images in the folders and their corresponding labels
        for label, folder in enumerate(os.listdir(root_dir)):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path):
                  if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
                    self.samples.append((os.path.join(folder_path, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        try:
            image_path, label = self.samples[index]
            image = Image.open(image_path).convert('RGB')

            if self.transform:
                image = self.transform(image)

            return image, label

        except Exception as e:
            # If there's an error opening or processing the image, print the error and return None
            #print(f"Error loading image: {image_path} - {e}")
            return self.__getitem__(index + 1)

In [13]:
# Instantiate custom dataset
root_dir = '/content/animal_data'
custom_dataset = CustomImageDataset(root_dir, transform=transform)

In [14]:
len(custom_dataset)

1944

In [15]:
# Define the size of the training set
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size

# Split the dataset into training and testing sets
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

len(train_dataset), len(test_dataset)

(1555, 389)

In [16]:
# Define batch size for DataLoader
batch_size = 16

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create DataLoader for testing set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
# Load pre-trained model
model = timm.create_model('resnet50', pretrained=True)

# Modify the final layer
num_classes = 15
model.fc = nn.Linear(model.fc.in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [18]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [19]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                             lr=0.01)

In [22]:
def train(dataloader, model, loss_fn, optimizer,device):
    size = len(dataloader.dataset)
    model.to(device)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    model.to(device)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
start_time = time.time()


# Train and test model
epochs = 5
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train(train_loader, model, loss_fn, optimizer,device)
    test(test_loader, model, loss_fn,device)

current_time = time.time()
total = current_time - start_time
print("Done!")
print(f'Training Took: {total/60} minutes!')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
---------
loss: 2.675511  [   16/ 1555]
loss: 2.619143  [  176/ 1555]
loss: 2.590198  [  336/ 1555]
loss: 2.557163  [  496/ 1555]
loss: 2.622115  [  656/ 1555]
loss: 2.617106  [  816/ 1555]
loss: 2.544883  [  976/ 1555]
loss: 2.509099  [ 1136/ 1555]
loss: 2.472153  [ 1296/ 1555]
loss: 2.539844  [ 1456/ 1555]
Test Error: 
 Accuracy: 70.2%, Avg loss: 2.504012 

Epoch: 1
---------
loss: 2.476554  [   16/ 1555]
loss: 2.427439  [  176/ 1555]
loss: 2.488664  [  336/ 1555]
loss: 2.404421  [  496/ 1555]
loss: 2.392542  [  656/ 1555]
loss: 2.347797  [  816/ 1555]
loss: 2.408553  [  976/ 1555]
loss: 2.304940  [ 1136/ 1555]
loss: 2.318841  [ 1296/ 1555]
loss: 2.381348  [ 1456/ 1555]
Test Error: 
 Accuracy: 72.5%, Avg loss: 2.419537 

Epoch: 2
---------
loss: 2.147450  [   16/ 1555]
loss: 2.246768  [  176/ 1555]
loss: 2.190455  [  336/ 1555]
loss: 2.301937  [  496/ 1555]
loss: 2.189370  [  656/ 1555]
loss: 2.127033  [  816/ 1555]
loss: 2.014006  [  976/ 1555]
loss: 2.038194  [ 1136/ 1555]

In [24]:
timm.list_models("convnext*")

['convnext_atto',
 'convnext_atto_ols',
 'convnext_base',
 'convnext_femto',
 'convnext_femto_ols',
 'convnext_large',
 'convnext_large_mlp',
 'convnext_nano',
 'convnext_nano_ols',
 'convnext_pico',
 'convnext_pico_ols',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge',
 'convnext_xxlarge',
 'convnextv2_atto',
 'convnextv2_base',
 'convnextv2_femto',
 'convnextv2_huge',
 'convnextv2_large',
 'convnextv2_nano',
 'convnextv2_pico',
 'convnextv2_small',
 'convnextv2_tiny']

timm models : https://www.kaggle.com/code/jhoward/which-image-models-are-best/